Movie theater with one ticket counter, selling tickets for three movies.

* People arrive randomly
* People try to buy a random number (1-6) of tickets for a random movie
* When a movie is sold out, all people waiting to buy tickets for that movie leave the queue


A moviegoer
* begins waiting until it's his turn or until the sold out signal is triggered
* if he gets to counter, he tries to buy tickets
  * if there are enough tickets, he is successful
  * otherwise he leaves
* if there are <2 tickets remaining, the sold out signal is triggered

In [6]:
import random
import simpy

from typing import NamedTuple

RANDOM_SEED = 42
TICKETS = 50  # avail tickets per movie
SELLOUT_THRESHOLD = 2
SIM_TIME = 120


class Theater(NamedTuple):
    counter: simpy.Resource
    movies: list[str]
    available: dict[str, int]
    sold_out: dict[str, simpy.Event]
    when_sold_out: dict[str, float | None]
    num_renegers: dict[str, int]


def moviegoer(env: simpy.Environment, movie, num_tickets: int, theater: Theater):
    """Moviegoer attempts to buy a number of tickets for a certain movie in a theater.
    
    """
    with theater.counter.request() as my_turn:
        # wait until it's my turn or until movie sells out
        result = yield my_turn | theater.sold_out[movie]

        # check if it's my turn or if sold out
        if my_turn not in result:
            # must've sold out
            theater.num_renegers[movie] += 1
            return

        # check if enough tickets left
        if theater.available[movie] < num_tickets:
            # moviegoer leaves counter after being disappointed
            yield env.timeout(0.5)
            return

        # buy tickets
        theater.available[movie] -= num_tickets
        # if we are now sold out, send sold out signal
        if theater.available[movie] < SELLOUT_THRESHOLD:
            theater.sold_out[movie].succeed()  # why not just a bool?
            theater.when_sold_out[movie] = env.now
            theater.available[movie] = 0
        # leave counter after paying
        yield env.timeout(1)


def customer_arrivals(env: simpy.Environment, theater):
    """Process: create new moviegoers until sim ends"""
    while True:
        # Scheduling next arrival event
        yield env.timeout(random.expovariate(1 / 0.5))

        movie = random.choice(theater.movies)
        num_tickets = random.randint(1, 6)
        if theater.available[movie]:
            env.process(moviegoer(env, movie, num_tickets, theater))


print("Movie Renege")
random.seed(RANDOM_SEED)

env = simpy.Environment()

# create theater
movies = ['Python Unchained', 'Kill Process', 'Pulp Implementation']
theater = Theater(
    counter=simpy.Resource(env, capacity=1),
    movies=movies,
    available={movie: TICKETS for movie in movies},
    sold_out={movie: env.event() for movie in movies},
    when_sold_out={movie: None for movie in movies},
    num_renegers={movie: 0 for movie in movies},
)

# start process and run
env.process(customer_arrivals(env, theater))
env.run(until=SIM_TIME)

# Analysis
for movie in movies:
    if theater.sold_out[movie]:
        sellout_time = theater.when_sold_out[movie]
        num_renegers = theater.num_renegers[movie]
        print(f"Movie {movie!r} sold out after {sellout_time:.1f} minutes.")
        print(f"  Number of people disappointed in queue: {num_renegers}")

Movie Renege
Movie 'Python Unchained' sold out after 38.0 minutes.
  Number of people disappointed in queue: 16
Movie 'Kill Process' sold out after 43.0 minutes.
  Number of people disappointed in queue: 5
Movie 'Pulp Implementation' sold out after 28.0 minutes.
  Number of people disappointed in queue: 5


: 